• v /gaze/gaze_exports.zip najdes exporty z Tobii Studia pre kazdeho participanta zvlast   
• v /gaze/seg_files su opisane zaciatky a konce segmentov pre jednotlivych participantov. Hodnoty v subore su zo stlpca `RecordingTimestamp`   
• v /screens/BT_MSNV_withIDs najdes screenshoty obrazoviek s IDckami, ktore su aj v seg filoch   
• v /csv/gaze_with_UCs_BT.csv najdes metriky vypocitane zo sledovania pohladu pre kazdeho participanta a kazdy citany dokument + charakteristiky pouzivatelov + cas a presnost vykonavania uloh   

In [1]:
#%run Functions.ipynb

from os import listdir
from os.path import isfile, join

import pandas as pd
import numpy as np

In [3]:
def parse_tsv_by_screen():
    columns = data_mmd.columns
    data = {
        3 : pd.DataFrame(columns=columns),
        5 : pd.DataFrame(columns=columns),
        9 : pd.DataFrame(columns=columns),
        11 : pd.DataFrame(columns=columns),
        18 : pd.DataFrame(columns=columns),
        20 : pd.DataFrame(columns=columns),
        27 : pd.DataFrame(columns=columns),
        28 : pd.DataFrame(columns=columns),
        30 : pd.DataFrame(columns=columns),
        60 : pd.DataFrame(columns=columns),
        62 : pd.DataFrame(columns=columns),
        66 : pd.DataFrame(columns=columns),
        72 : pd.DataFrame(columns=columns),
        74 : pd.DataFrame(columns=columns),
        76 : pd.DataFrame(columns=columns)
    }
    
    for i in range(0, len(gaze_exports_names)):
        participant_seg_files = pd.read_csv(path + segfiles_path + seg_files_names[i], low_memory=False, sep=",")
        participant = f = pd.read_csv(path + gazes_path + gaze_exports_names[i], low_memory=False, sep="\t")
        print("Working on " + participant.iloc[0]['ParticipantName'])
        for ide,row in participant_seg_files.iterrows():
            mmd_id = row['mmd_id']
            start = row['start']
            end = row['end']
            print("--" + str(mmd_id))

            data_mmd = participant[(participant['RecordingTimestamp'] >= start) & (participant['RecordingTimestamp'] <= end)]
            data[mmd_id] = data[mmd_id].append(data_mmd)
        print("--------------------------")  
        i = i + 1
        
    for d in data:
        data[d].to_csv(path + data_by_screen_path + str(d) +".tsv",sep="\t")
        
        
def load_aois(aois_index):
    all_aois = {}

    aois_columns = ["Typ","1","2","3","4"]
    aois_file = pd.read_csv(path + aois_path + aois_files_names[aois_index], sep="\t").iloc[:, : 5]
    aois_file.columns = aois_columns

    i = 0
    for index, row in aois_file.iterrows():
        a = Point(float(aois_file.iloc[i][1].split(",")[0]), float(aois_file.iloc[i][1].split(",")[1]))
        b = Point(float(aois_file.iloc[i][2].split(",")[0]), float(aois_file.iloc[i][2].split(",")[1]))
        c = Point(float(aois_file.iloc[i][3].split(",")[0]), float(aois_file.iloc[i][3].split(",")[1]))
        d = Point(float(aois_file.iloc[i][4].split(",")[0]), float(aois_file.iloc[i][4].split(",")[1]))

        poly = Polygon(((a.x,a.y),(b.x,b.y),(c.x,c.y),(d.x,d.y)))

        all_aois[aois_names[i]] = poly

        i = i + 1
        
    return all_aois

In [2]:
path = "../MSVN-Data/"
gazes_path = "gaze_exports/"
segfiles_path = "seg_files/"
aois_path = "aois/"
data_by_screen_path = "data_by_screens/"

metrics_file = "gaze_with_UCs_BT.csv"


aois_names = ["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","R","S","T","U","V","X","Y"]

In [3]:
gaze_exports_names = [f for f in listdir(path + gazes_path) if isfile(join(path + gazes_path, f))][:-4]
seg_files_names = [f for f in listdir(path + segfiles_path) if isfile(join(path + segfiles_path, f))]
aois_files_names = [f for f in listdir(path + aois_path) if isfile(join(path + aois_path, f))]
databyscreens_names = [f for f in listdir(path + data_by_screen_path) if isfile(join(path + data_by_screen_path, f))]

In [28]:
# Get all participansts, their names and create instances
ALL_PARTICIPANTS = {}
ALL_PARTICIPANTS_FEATURES_NAMES = []
ALL_PARTICIPANTS_NAMES = [] 
ALL_PARTICIPANTS_AFTER_COMPLETE_CHECK_NAMES = []

DONT_HAVE_ENOUGH_DATA = []
DONT_COMPLETE_TASK = []
DONT_PASS_CALIBRATION = []
DONT_PASS_VALIDATION = []




In [4]:
#DATASET = pd.read_csv(path + data_by_screen_path + databyscreens_names[task_index], low_memory=False, sep="\t")
task_index = 1
DATASET = pd.read_csv(path + gazes_path + gaze_exports_names[task_index], low_memory=False, sep="\t")
#AOIS = load_aois(task_index)

# Participanti, ktori nesplnili ulohy (prevzal som to od Mokreho ako to mal on)
#DONT_COMPLETE_TASK = CONFIG_PARSER.get('delete', TASK_NUMBER).split("\n")
ALL_PARTICIPANTS_NAMES = DATASET['ParticipantName'].unique()

#for tester in ALL_PARTICIPANTS_NAMES:
    #ALL_PARTICIPANTS[tester] = Participant(tester)

In [12]:
#ALL_PARTICIPANTS_FEATURES_NAMES = set_participants_features(True, False)

In [41]:
DATASET.columns

Index(['ExportDate', 'StudioVersionRec', 'StudioProjectName', 'StudioTestName',
       'ParticipantName', 'RecordingName', 'RecordingDate',
       'RecordingDuration', 'RecordingResolution', 'PresentationSequence',
       'FixationFilter', 'MediaName', 'MediaPosX (ADCSpx)',
       'MediaPosY (ADCSpx)', 'MediaWidth', 'MediaHeight', 'SegmentName',
       'SegmentStart', 'SegmentEnd', 'SegmentDuration', 'SceneName',
       'SceneSegmentStart', 'SceneSegmentEnd', 'SceneSegmentDuration',
       'RecordingTimestamp', 'LocalTimeStamp', 'EyeTrackerTimestamp',
       'MouseEventIndex', 'MouseEvent', 'MouseEventX (ADCSpx)',
       'MouseEventY (ADCSpx)', 'MouseEventX (MCSpx)', 'MouseEventY (MCSpx)',
       'KeyPressEventIndex', 'KeyPressEvent', 'StudioEventIndex',
       'StudioEvent', 'StudioEventData', 'ExternalEventIndex', 'ExternalEvent',
       'ExternalEventValue', 'EventMarkerValue', 'FixationIndex',
       'SaccadeIndex', 'GazeEventType', 'GazeEventDuration',
       'FixationPointX (MCSp

In [9]:
all_screen_names = DATASET['StudioEventData'].dropna().unique()[:-4]

In [10]:
all_screen_names

array(['V1_190x120.png', 'V2_950x120.png', 'V3_1710x120.png',
       'V4_190x600.png', 'V5_950x600.png', 'V6_1710x600.png',
       'V7_190x1080.png', 'V8_950x1080.png', 'V9_1710x1080.png'],
      dtype=object)

In [ ]:
DATASET[DATASET['StudioEventData'] == "VA_950x600.png"]

In [52]:
bum = DATASET[DATASET['StudioEventData'] == "VD_950x600.png"]
bum

,ExportDate,StudioVersionRec,StudioProjectName,StudioTestName,ParticipantName,RecordingName,RecordingDate,RecordingDuration,RecordingResolution,PresentationSequence,...,DistanceLeft,DistanceRight,PupilLeft,PupilRight,ValidityLeft,ValidityRight,IRMarkerCount,IRMarkerID,PupilGlassesRight,Unnamed: 85
5222,7.8.2018,3.4.8,IN MSNV Study,New test,P201,Rec 201,15.5.2018,1806273,1920 x 1200,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5643,7.8.2018,3.4.8,IN MSNV Study,New test,P201,Rec 201,15.5.2018,1806273,1920 x 1200,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
DATASET[(DATASET.index >= 5222) & (DATASET.index <= 5643)]

,ExportDate,StudioVersionRec,StudioProjectName,StudioTestName,ParticipantName,RecordingName,RecordingDate,RecordingDuration,RecordingResolution,PresentationSequence,...,DistanceLeft,DistanceRight,PupilLeft,PupilRight,ValidityLeft,ValidityRight,IRMarkerCount,IRMarkerID,PupilGlassesRight,Unnamed: 85
5222,7.8.2018,3.4.8,IN MSNV Study,New test,P201,Rec 201,15.5.2018,1806273,1920 x 1200,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5223,7.8.2018,3.4.8,IN MSNV Study,New test,P201,Rec 201,15.5.2018,1806273,1920 x 1200,NaN,...,686.91,686.91,2.92,2.93,0.0,0.0,NaN,NaN,NaN,NaN
5224,7.8.2018,3.4.8,IN MSNV Study,New test,P201,Rec 201,15.5.2018,1806273,1920 x 1200,NaN,...,686.91,686.91,2.86,3.02,0.0,0.0,NaN,NaN,NaN,NaN
5225,7.8.2018,3.4.8,IN MSNV Study,New test,P201,Rec 201,15.5.2018,1806273,1920 x 1200,NaN,...,686.89,686.89,2.92,2.96,0.0,0.0,NaN,NaN,NaN,NaN
5226,7.8.2018,3.4.8,IN MSNV Study,New test,P201,Rec 201,15.5.2018,1806273,1920 x 1200,NaN,...,686.77,686.77,2.90,2.96,0.0,0.0,NaN,NaN,NaN,NaN
5227,7.8.2018,3.4.8,IN MSNV Study,New test,P201,Rec 201,15.5.2018,1806273,1920 x 1200,NaN,...,686.76,686.76,2.90,2.97,0.0,0.0,NaN,NaN,NaN,NaN
5228,7.8.2018,3.4.8,IN MSNV Study,New test,P201,Rec 201,15.5.2018,1806273,1920 x 1200,NaN,...,686.76,686.76,2.89,2.97,0.0,0.0,NaN,NaN,NaN,NaN
5229,7.8.2018,3.4.8,IN MSNV Study,New test,P201,Rec 201,15.5.2018,1806273,1920 x 1200,NaN,...,686.79,686.79,2.84,3.00,0.0,0.0,NaN,NaN,NaN,NaN
5230,7.8.2018,3.4.8,IN MSNV Study,New test,P201,Rec 201,15.5.2018,1806273,1920 x 1200,NaN,...,686.84,686.84,2.89,3.12,0.0,0.0,NaN,NaN,NaN,NaN
5231,7.8.2018,3.4.8,IN MSNV Study,New test,P201,Rec 201,15.5.2018,1806273,1920 x 1200,NaN,...,686.96,686.96,2.89,2.94,0.0,0.0,NaN,NaN,NaN,NaN


In [11]:
class Participant():
    
    def __init__(self):
        self.data = {}
        for sc in all_screen_names:
            self.data[sc] = None
        
        
    def set_screen(self, screenName, data):
        self.data[screenName] = data
        
        


In [12]:
ALL_PARTICIPANTS = {}

In [13]:
task_index = 0
for gaze_export_name in gaze_exports_names:
    print("Working on " + gaze_export_name)
    DATASET = pd.read_csv(path + gazes_path + gaze_export_name, low_memory=False, sep="\t")

    testerName = gaze_export_name[-7:][:-4]
    ALL_PARTICIPANTS[testerName] = Participant()

    for sc in all_screen_names:
        print(".......... " + sc)
        sc_data = DATASET[DATASET['StudioEventData'] == sc]
        lower = sc_data.index[0]
        upper = sc_data.index[1]
        data = DATASET[(DATASET.index >= lower) & (DATASET.index <= upper)]
        ALL_PARTICIPANTS[testerName].set_screen(sc, data)

Working on IN MSNV Study_Rec 200.tsv
.......... V1_190x120.png
.......... V2_950x120.png
.......... V3_1710x120.png
.......... V4_190x600.png
.......... V5_950x600.png
.......... V6_1710x600.png
.......... V7_190x1080.png
.......... V8_950x1080.png
.......... V9_1710x1080.png
Working on IN MSNV Study_Rec 201.tsv
.......... V1_190x120.png
.......... V2_950x120.png
.......... V3_1710x120.png
.......... V4_190x600.png
.......... V5_950x600.png
.......... V6_1710x600.png
.......... V7_190x1080.png
.......... V8_950x1080.png
.......... V9_1710x1080.png
Working on IN MSNV Study_Rec 202.tsv
.......... V1_190x120.png
.......... V2_950x120.png
.......... V3_1710x120.png
.......... V4_190x600.png
.......... V5_950x600.png
.......... V6_1710x600.png
.......... V7_190x1080.png
.......... V8_950x1080.png
.......... V9_1710x1080.png
Working on IN MSNV Study_Rec 203.tsv
.......... V1_190x120.png
.......... V2_950x120.png
.......... V3_1710x120.png
.......... V4_190x600.png
.......... V5_950x600.png
.

.......... V1_190x120.png
.......... V2_950x120.png
.......... V3_1710x120.png
.......... V4_190x600.png
.......... V5_950x600.png
.......... V6_1710x600.png
.......... V7_190x1080.png
.......... V8_950x1080.png
.......... V9_1710x1080.png
Working on IN MSNV Study_Rec 235.tsv
.......... V1_190x120.png
.......... V2_950x120.png
.......... V3_1710x120.png
.......... V4_190x600.png
.......... V5_950x600.png
.......... V6_1710x600.png
.......... V7_190x1080.png
.......... V8_950x1080.png
.......... V9_1710x1080.png
Working on IN MSNV Study_Rec 236.tsv
.......... V1_190x120.png
.......... V2_950x120.png
.......... V3_1710x120.png
.......... V4_190x600.png
.......... V5_950x600.png
.......... V6_1710x600.png
.......... V7_190x1080.png
.......... V8_950x1080.png
.......... V9_1710x1080.png
Working on IN MSNV Study_Rec 237.tsv
.......... V1_190x120.png
.......... V2_950x120.png
.......... V3_1710x120.png
.......... V4_190x600.png
.......... V5_950x600.png
.......... V6_1710x600.png
.......... 

In [24]:
all_names = ALL_PARTICIPANTS.keys()

In [25]:
data = {}
for sc in all_screen_names:
    data[sc] = pd.DataFrame()

In [29]:
for name in all_names:
    print("Working on " + name)
    for sc in all_screen_names:
        print("........... " + sc)
        data[sc] = data[sc].append(ALL_PARTICIPANTS[name].data[sc])

Working on 200
........... V1_190x120.png
........... V2_950x120.png
........... V3_1710x120.png
........... V4_190x600.png
........... V5_950x600.png
........... V6_1710x600.png
........... V7_190x1080.png
........... V8_950x1080.png
........... V9_1710x1080.png
Working on 201
........... V1_190x120.png
........... V2_950x120.png
........... V3_1710x120.png
........... V4_190x600.png
........... V5_950x600.png
........... V6_1710x600.png
........... V7_190x1080.png
........... V8_950x1080.png
........... V9_1710x1080.png
Working on 202
........... V1_190x120.png
........... V2_950x120.png
........... V3_1710x120.png
........... V4_190x600.png
........... V5_950x600.png
........... V6_1710x600.png
........... V7_190x1080.png
........... V8_950x1080.png
........... V9_1710x1080.png
Working on 203
........... V1_190x120.png
........... V2_950x120.png
........... V3_1710x120.png
........... V4_190x600.png
........... V5_950x600.png
........... V6_1710x600.png
........... V7_190x1080.png
.

........... V6_1710x600.png
........... V7_190x1080.png
........... V8_950x1080.png
........... V9_1710x1080.png
Working on 236
........... V1_190x120.png
........... V2_950x120.png
........... V3_1710x120.png
........... V4_190x600.png
........... V5_950x600.png
........... V6_1710x600.png
........... V7_190x1080.png
........... V8_950x1080.png
........... V9_1710x1080.png
Working on 237
........... V1_190x120.png
........... V2_950x120.png
........... V3_1710x120.png
........... V4_190x600.png
........... V5_950x600.png
........... V6_1710x600.png
........... V7_190x1080.png
........... V8_950x1080.png
........... V9_1710x1080.png
Working on 238
........... V1_190x120.png
........... V2_950x120.png
........... V3_1710x120.png
........... V4_190x600.png
........... V5_950x600.png
........... V6_1710x600.png
........... V7_190x1080.png
........... V8_950x1080.png
........... V9_1710x1080.png
Working on 239
........... V1_190x120.png
........... V2_950x120.png
........... V3_1710x120.pn

In [30]:
data

{'V1_190x120.png':     ExportDate StudioVersionRec StudioProjectName StudioTestName  \
 76    7.8.2018            3.4.8     IN MSNV Study       New test   
 77    7.8.2018            3.4.8     IN MSNV Study       New test   
 78    7.8.2018            3.4.8     IN MSNV Study       New test   
 79    7.8.2018            3.4.8     IN MSNV Study       New test   
 80    7.8.2018            3.4.8     IN MSNV Study       New test   
 81    7.8.2018            3.4.8     IN MSNV Study       New test   
 82    7.8.2018            3.4.8     IN MSNV Study       New test   
 83    7.8.2018            3.4.8     IN MSNV Study       New test   
 84    7.8.2018            3.4.8     IN MSNV Study       New test   
 85    7.8.2018            3.4.8     IN MSNV Study       New test   
 86    7.8.2018            3.4.8     IN MSNV Study       New test   
 87    7.8.2018            3.4.8     IN MSNV Study       New test   
 88    7.8.2018            3.4.8     IN MSNV Study       New test   
 89    7.8.2018 

In [32]:
for d in data:
    data[d].to_csv(path + "calibration_data/" + str(d[:-4]) +".tsv",sep="\t")